In [1]:
from nfstream import NFStreamer
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
from sklearn.preprocessing import MinMaxScaler
from menelaus.data_drift import KdqTreeBatch

# df_withCD = NFStreamer(source="../pcaps/withCD.pcapng", n_dissections=0, statistical_analysis=True).to_pandas()
# df_woCD = NFStreamer(source="../pcaps/woCD.pcapng", n_dissections=0, statistical_analysis=True).to_pandas()
# df_test_new = NFStreamer(source="../pcaps/test_271022.pcap", n_dissections=0, statistical_analysis=True).to_pandas()

# print(df_withCD.shape)
# print(df_woCD.shape)
# print(df_test_new.shape)
list_dropcols = ['id', 'expiration_id', 'src_ip', 'src_mac', 
                 'src_oui', 'src_port', 'dst_ip', 
                 'dst_mac', 'dst_oui', 'dst_port', 
                 'protocol', 'ip_version', 'vlan_id', 'tunnel_id',
                 'bidirectional_first_seen_ms', 'bidirectional_last_seen_ms',
                 'src2dst_first_seen_ms', 'src2dst_last_seen_ms', 
                 'dst2src_first_seen_ms', 'dst2src_last_seen_ms']

In [ ]:
df_woCD = df_woCD.drop(columns=list_dropcols)
df_withCD = df_withCD.drop(columns=list_dropcols)
df_test_new = df_test_new.drop(columns=list_dropcols)

In [ ]:
scaler = MinMaxScaler()
df_woCD[df_woCD.columns] = scaler.fit_transform(df_woCD[df_woCD.columns])
df_withCD[df_withCD.columns] = scaler.fit_transform(df_withCD[df_withCD.columns])
df_test_new[df_test_new.columns] = scaler.fit_transform(df_test_new[df_test_new.columns])

In [3]:
det = KdqTreeBatch(alpha=0.1)
# det.set_reference(df_woCD)

In [ ]:
det.update(df_test_new)

In [ ]:
det.update(df_test_new)

In [ ]:
print(det.drift_state)

In [4]:
cic_df = pd.read_csv("../CICIDS_Monday_preprocessed.csv")

In [5]:
cic_df.shape

(355732, 70)

In [ ]:
print(min(cic_df.bidirectional_first_seen_ms))

In [6]:
col_key = 'bidirectional_first_seen_ms'
win_count = 10
win_sz = (max(cic_df[col_key])-min(cic_df[col_key]))/win_count
start_time = min(cic_df[col_key])
scaler = MinMaxScaler()
list_dropcols_2 = ['bidirectional_first_seen_ms', 'bidirectional_last_seen_ms',
                 'src2dst_first_seen_ms', 'src2dst_last_seen_ms', 
                 'dst2src_first_seen_ms', 'dst2src_last_seen_ms']
end_time = start_time + win_sz*2
df_win1 = cic_df[(cic_df[col_key] >= start_time) & (cic_df[col_key] <= end_time)]
df_win1 = df_win1.drop(columns=list_dropcols_2)
df_win1[df_win1.columns] = scaler.fit_transform(df_win1[df_win1.columns])
det.set_reference(df_win1)
# start_time = end_time
for i in range(1,win_count+1):
    end_time = start_time + win_sz
    df_win = cic_df[(cic_df[col_key] >= start_time) & (cic_df[col_key] <= end_time)]
    df_win = df_win.drop(columns=list_dropcols_2)
    try:
        df_win[df_win.columns] = scaler.fit_transform(df_win[df_win.columns])
    except:
        start_time = end_time
        print(f"No sample between {start_time} and  {end_time}")
        continue

    det.update(df_win)
    print("Segment No.:", i, "Drift State:", det.drift_state, "Segment Shape:", df_win.shape)
    start_time = end_time
    # det.set_reference(df_win1)
    det.update(df_win1)

Segment No.: 1 Drift State: drift Segment Shape: (33722, 64)
Segment No.: 2 Drift State: drift Segment Shape: (56006, 64)
Segment No.: 3 Drift State: drift Segment Shape: (40298, 64)
Segment No.: 4 Drift State: drift Segment Shape: (42273, 64)
Segment No.: 5 Drift State: drift Segment Shape: (24600, 64)
Segment No.: 6 Drift State: drift Segment Shape: (27462, 64)
Segment No.: 7 Drift State: drift Segment Shape: (37366, 64)
Segment No.: 8 Drift State: drift Segment Shape: (28303, 64)
Segment No.: 9 Drift State: drift Segment Shape: (27518, 64)
Segment No.: 10 Drift State: drift Segment Shape: (38181, 64)


In [ ]:
max(cic_df[col_key])

In [ ]:
(max(cic_df[col_key])-min(cic_df[col_key]))/win_count